START OF FILE

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("music_genre.csv")

In [3]:
data.head()

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic


In [4]:
from sklearn.neural_network import MLPClassifier
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
print(data.shape)

(50005, 18)


In [5]:
print(data.shape)
data.describe().transpose()

(50005, 18)


,count,mean,std,min,25%,50%,75%,max
instance_id,50000.0,55888.396360,20725.256253,20002.000000,37973.5000,55913.500000,73863.250000,91759.000
popularity,50000.0,44.220420,15.542008,0.000000,34.0000,45.000000,56.000000,99.000
acousticness,50000.0,0.306383,0.341340,0.000000,0.0200,0.144000,0.552000,0.996
danceability,50000.0,0.558241,0.178632,0.059600,0.4420,0.568000,0.687000,0.986
duration_ms,50000.0,221252.602860,128671.957157,-1.000000,174800.0000,219281.000000,268612.250000,4830606.000
energy,50000.0,0.599755,0.264559,0.000792,0.4330,0.643000,0.815000,0.999
instrumentalness,50000.0,0.181601,0.325409,0.000000,0.0000,0.000158,0.155000,0.996
liveness,50000.0,0.193896,0.161637,0.009670,0.0969,0.126000,0.244000,1.000
loudness,50000.0,-9.133761,6.162990,-47.046000,-10.8600,-7.276500,-5.173000,3.744
speechiness,50000.0,0.093586,0.101373,0.022300,0.0361,0.048900,0.098525,0.942


In [6]:
size = len(data.index)
print(size)

50005


Next steps are to remove useless features (instance_id, artist name, track name, and obtained date) and change genre to 0 or 1 if it is rock music or not

In [7]:
data = data.drop(columns=['instance_id', 'duration_ms', "artist_name", "track_name", "obtained_date", "tempo"])

#data = data.drop(columns=['track_name', 'obtained_date'])
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
popularity,50000.0,44.220420,15.542008,0.000000,34.0000,45.000000,56.000000,99.000
acousticness,50000.0,0.306383,0.341340,0.000000,0.0200,0.144000,0.552000,0.996
danceability,50000.0,0.558241,0.178632,0.059600,0.4420,0.568000,0.687000,0.986
energy,50000.0,0.599755,0.264559,0.000792,0.4330,0.643000,0.815000,0.999
instrumentalness,50000.0,0.181601,0.325409,0.000000,0.0000,0.000158,0.155000,0.996
liveness,50000.0,0.193896,0.161637,0.009670,0.0969,0.126000,0.244000,1.000
loudness,50000.0,-9.133761,6.162990,-47.046000,-10.8600,-7.276500,-5.173000,3.744
speechiness,50000.0,0.093586,0.101373,0.022300,0.0361,0.048900,0.098525,0.942
valence,50000.0,0.456264,0.247119,0.000000,0.2570,0.448000,0.648000,0.992


In [8]:
size = len(data.index)

for x in range(size):
    temp = data.at[x, "music_genre"]
    if temp == "Electronic":
        data.at[x, "music_genre"] = 1.0
    else: 
        data.at[x, "music_genre"] = 0.0


In [9]:
data.head()


,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,valence,music_genre
0,27.0,0.00468,0.652,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,0.759,1.0
1,31.0,0.01270,0.622,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,0.531,1.0
2,28.0,0.00306,0.620,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,0.333,1.0
3,34.0,0.02540,0.774,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,0.270,1.0
4,32.0,0.00465,0.638,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,0.323,1.0


Change music_genre from object to int or float or boolean

In [10]:
#data.astype({"music_genre" : "float64"}).dtypes

data["music_genre"] = pd.to_numeric(data["music_genre"])
print(data.dtypes)

popularity          float64
acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
valence             float64
music_genre         float64
dtype: object


Change keys to numbers 1-12

In [11]:
for x in range(size):
    temp = data.at[x, "key"]
    if temp == "A":
        data.at[x, "key"] = 1.0
    elif temp == "A#":
        data.at[x, "key"] = 2.0
    elif temp == "B":
        data.at[x, "key"] = 3.0
    elif temp == "C":
        data.at[x, "key"] = 4.0
    elif temp == "C#":
        data.at[x, "key"] = 5.0
    elif temp == "D":
        data.at[x, "key"] = 6.0
    elif temp == "D#":
        data.at[x, "key"] = 7.0
    elif temp == "E":
        data.at[x, "key"] = 8.0
    elif temp == "F":
        data.at[x, "key"] = 9.0
    elif temp == "F#":
        data.at[x, "key"] = 10.0
    elif temp == "G":
        data.at[x, "key"] = 11.0
    else:
        data.at[x, "key"] = 12.0
    
    



In [12]:
data.head()

,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,valence,music_genre
0,27.0,0.00468,0.652,0.941,0.79200,2.0,0.115,-5.201,Minor,0.0748,0.759,1.0
1,31.0,0.01270,0.622,0.890,0.95000,6.0,0.124,-7.043,Minor,0.0300,0.531,1.0
2,28.0,0.00306,0.620,0.755,0.01180,12.0,0.534,-4.617,Major,0.0345,0.333,1.0
3,34.0,0.02540,0.774,0.700,0.00253,5.0,0.157,-4.498,Major,0.2390,0.270,1.0
4,32.0,0.00465,0.638,0.587,0.90900,10.0,0.157,-6.266,Major,0.0413,0.323,1.0


In [13]:
data["key"] = pd.to_numeric(data["key"])
print(data.dtypes)

popularity          float64
acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                 float64
liveness            float64
loudness            float64
mode                 object
speechiness         float64
valence             float64
music_genre         float64
dtype: object


Change mode to 1 for major and 0 for minor

In [14]:
for x in range(size):
    temp = data.at[x, "mode"]
    if temp == "Major":
        data.at[x, "mode"] = 1.0
    else: 
        data.at[x, "mode"] = 0.0
data["mode"] = pd.to_numeric(data["mode"])
print(data.dtypes)

popularity          float64
acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                 float64
liveness            float64
loudness            float64
mode                float64
speechiness         float64
valence             float64
music_genre         float64
dtype: object


In [15]:
print(data.shape)

(50005, 12)


In [16]:
size = len(data.index)

minimum = data['loudness'].min() * -1


for x in range(size):
    data.at[x, 'loudness'] = data.at[x, 'loudness'] + minimum

data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
popularity,50000.0,44.220420,15.542008,0.000000,34.0000,45.000000,56.000000,99.000
acousticness,50000.0,0.306383,0.341340,0.000000,0.0200,0.144000,0.552000,0.996
danceability,50000.0,0.558241,0.178632,0.059600,0.4420,0.568000,0.687000,0.986
energy,50000.0,0.599755,0.264559,0.000792,0.4330,0.643000,0.815000,0.999
instrumentalness,50000.0,0.181601,0.325409,0.000000,0.0000,0.000158,0.155000,0.996
key,50005.0,6.354985,3.457513,1.000000,4.0000,6.000000,9.000000,12.000
liveness,50000.0,0.193896,0.161637,0.009670,0.0969,0.126000,0.244000,1.000
loudness,50000.0,37.912239,6.162990,0.000000,36.1860,39.769500,41.873000,50.790
mode,50005.0,0.641916,0.479442,0.000000,0.0000,1.000000,1.000000,1.000
speechiness,50000.0,0.093586,0.101373,0.022300,0.0361,0.048900,0.098525,0.942


Next step is to delete entries with 0 or below duration and adjust loudness to positive scale

Time to normalize

In [17]:
target = ['music_genre']
predictors = ['popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'valence']

In [18]:
outSize = len(predictors)


for x in range(outSize):
    inSize = len(data[predictors[x]])
    temp = predictors[x]
    maximum = data[temp].max()
    for y in range(inSize):
        data.at[y, temp] = data.at[y, temp] / maximum
    

Get rid of empties

In [19]:
data = data.dropna()
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
popularity,50000.0,0.446671,0.156990,0.000000,0.343434,0.454545,0.565657,1.0
acousticness,50000.0,0.307613,0.342711,0.000000,0.020080,0.144578,0.554217,1.0
danceability,50000.0,0.566167,0.181168,0.060446,0.448276,0.576065,0.696755,1.0
energy,50000.0,0.600355,0.264824,0.000793,0.433433,0.643644,0.815816,1.0
instrumentalness,50000.0,0.182330,0.326716,0.000000,0.000000,0.000159,0.155622,1.0
key,50000.0,0.529535,0.288102,0.083333,0.333333,0.500000,0.750000,1.0
liveness,50000.0,0.193896,0.161637,0.009670,0.096900,0.126000,0.244000,1.0
loudness,50000.0,0.746451,0.121343,0.000000,0.712463,0.783018,0.824434,1.0
mode,50000.0,0.641980,0.479423,0.000000,0.000000,1.000000,1.000000,1.0
speechiness,50000.0,0.099349,0.107615,0.023673,0.038323,0.051911,0.104591,1.0


Split data into testing and training

In [20]:
x = data[predictors].values
y = data[target].values
y = np.ravel(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state=40)

Make Classifier model

In [21]:
mlp = MLPClassifier(hidden_layer_sizes=(11,11,11), activation='relu', solver='adam', max_iter=500)
mlp.fit(x_train, y_train)




MLPClassifier(hidden_layer_sizes=(11, 11, 11), max_iter=500)

In [22]:
predict_train = mlp.predict(x_train)
predict_test = mlp.predict(x_test)

Now time to evaluate

In [23]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train, predict_train))

[[31096   440]
 [ 1905  1559]]
              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96     31536
         1.0       0.78      0.45      0.57      3464

    accuracy                           0.93     35000
   macro avg       0.86      0.72      0.77     35000
weighted avg       0.93      0.93      0.92     35000



Now evaluate with the test data

In [24]:
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test, predict_test))

[[13244   220]
 [  879   657]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     13464
         1.0       0.75      0.43      0.54      1536

    accuracy                           0.93     15000
   macro avg       0.84      0.71      0.75     15000
weighted avg       0.92      0.93      0.92     15000

